In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('../')
import test_cuda
from datetime import  datetime

In [47]:
device = test_cuda.device

In [48]:
epochs = 4
batch_size = 128
learning_rate = 0.01

In [49]:
transform = transforms.Compose(
[    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
    )

In [50]:
train_dataset = torchvision.datasets.CIFAR10(root='./data',train=True, download=True,transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False, download=True,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [51]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True,pin_memory=True)

In [52]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [53]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(3,6, 5)
        self.pool  = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16, 5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        self.relu = functional.relu

    def forward(self,x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [54]:
raise
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

RuntimeError: No active exception to reraise

In [55]:
n_total_steps = len(train_loader)

for epoch in range(2):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        loss = criterion(output,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i%2000) == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}")




Epoch [1/4], Step [1/391], Loss: 0.8571
Epoch [2/4], Step [1/391], Loss: 1.1082


In [80]:
model.eval()
with torch.inference_mode():
    n_corret = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        predicted = torch.argmax(output,1)
        n_samples+= labels.size(0)
        n_corret += (predicted == labels).sum().item()

        for i in range(labels.shape[0]):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    acc = 100 * n_corret/n_samples
    print(f"Accuracy of the network: {acc:.2f}")

    for i in range(10):
        acc = 100 * n_class_correct[i]/n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc:.2f}%')


Accuracy of the network: 61.19
Accuracy of plane: 64.80%
Accuracy of car: 83.00%
Accuracy of bird: 41.50%
Accuracy of cat: 43.70%
Accuracy of deer: 59.60%
Accuracy of dog: 45.20%
Accuracy of frog: 67.00%
Accuracy of horse: 77.90%
Accuracy of ship: 68.40%
Accuracy of truck: 60.80%


In [95]:
torch.save(model.state_dict(), datetime.today().strftime("./cifar_10_model_%Y_%m_%d_%H_%M_%S"))
